In [2]:
import cv2 
import os.path as osp
import os
import glob
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model,process_mmdet_results,vis_pose_result
import mmcv
import numpy as np
import shutil
def extract_frame(video_path):
    dname = 'temp'
    os.makedirs(dname, exist_ok=True)
    frame_tmpl = osp.join(dname, 'img_{:05d}.jpg')
    cap = cv2.VideoCapture(video_path)
    frame_paths = []
    cnt = 0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag:
            frame_path = frame_tmpl.format(cnt + 1)
            frame_paths.append(frame_path)
            cv2.imwrite(frame_path, frame)
            cnt += 1
        else: break
    cap.release()
    return frame_paths


c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
pose_config = 'mmpose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
pose_checkpoint = 'hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
###########
# det_config = 'Pose/yolox_s_8x8_300e_coco.py'
# det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'
##############
det_config = 'mmdetection/configs/yolox/yolox_s_8x8_300e_coco.py'
det_checkpoint = 'yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'

# initialize pose model
pose_model = init_pose_model(pose_config, pose_checkpoint)
# initialize detector
det_model = init_detector(det_config, det_checkpoint)

load checkpoint from local path: hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
load checkpoint from local path: yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth


In [4]:
files='VIDEO_TEST/VIDEO1.mp4'
frame_paths=extract_frame(files)

In [9]:
for frame in frame_paths:
    img = cv2.imread(frame)
    if cv2.waitKey(20)& 0xFF==ord('q'): break
    cv2.imshow('',img)
cv2.destroyAllWindows()

In [41]:
# model = init_pose_model(pose_config, pose_checkpoint, 'cuda')
# print('Performing Human Pose Estimation for each frame')
# prog_bar = mmcv.ProgressBar(len(frame_paths))

# num_frame = len(results)
# num_person = max([len(x) for x in results])
# kp = np.zeros((num_person, num_frame, 17, 3), dtype=np.float32)
# for i, (f, d) in enumerate(zip(frame_paths, results)):
#     # Align input format
#     d = [dict(bbox=x) for x in list(d) if x[-1] > 0.5]
#     pose = inference_top_down_pose_model(model, f, d, format='xyxy')[0]
#     for j, item in enumerate(pose):
#         kp[j, i] = item['keypoints']
#     prog_bar.update()


In [13]:
def detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr=0.5,device='cuda' ):
    model = init_detector(det_config, det_checkpoint, device)
    assert model.CLASSES[0] == 'person', ('We require you to use a detector '
                                          'trained on COCO')
    results = []
    print('Performing Human Detection for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))
    for frame_path in frame_paths:
        result = inference_detector(model, frame_path)
        # We only keep human detections with score larger than det_score_thr
        result = result[0][result[0][:, 4] >= det_score_thr]
        results.append(result)
        prog_bar.update()
    return results

def pose_inference(pose_config,pose_checkpoint, frame_paths, det_results, device='cuda'):
    model = init_pose_model(pose_config, pose_checkpoint, device)
    print('Performing Human Pose Estimation for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))

    num_frame = len(det_results)
    num_person = max([len(x) for x in det_results])
    kp = np.zeros((num_person, num_frame, 17, 3), dtype=np.float32)
    for i, (f, d) in enumerate(zip(frame_paths, det_results)):
        # Align input format
        # if len(d) == 0: 
        #     prog_bar.update()
        #     continue
        d = [dict(bbox=x) for x in list(d) if x[-1] > 0.5]
        pose = inference_top_down_pose_model(model, f, d, format='xyxy')[0]
        vis_ske = vis_pose_result(model,f,pose,dataset=model.cfg.data.test.type,show=False)
        if cv2.waitKey(20)& 0xFF==ord('q'): break
        cv2.imshow('',vis_ske)
        for j, item in enumerate(pose):
            kp[j, i] = item['keypoints']
        prog_bar.update()
    cv2.destroyAllWindows()
    return kp

def pose_extraction(vid,det_config, det_checkpoint,pose_config,pose_checkpoint,label, det_score_thr=0.5,device='cuda'):
    frame_paths = extract_frame(vid)
    det_results = detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr,device)
    image = cv2.imread(frame_paths[0])
    image_shape = (image.shape[0], image.shape[1])
    pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths, det_results, device)
    anno = dict()
    anno['keypoint'] = pose_results[..., :2]
    anno['keypoint_score'] = pose_results[..., 2]
    anno['frame_dir'] = osp.splitext(osp.basename(vid))[0]
    anno['img_shape'] = image_shape
    anno['original_shape'] = image_shape
    anno['total_frames'] = pose_results.shape[1]
    anno['label'] = label
    shutil.rmtree(osp.dirname(frame_paths[0]))
    return anno

In [11]:
model = init_detector(det_config, det_checkpoint)
assert model.CLASSES[0] == 'person', ('We require you to use a detector '
                                        'trained on COCO')
results = []
print('Performing Human Detection for each frame')
prog_bar = mmcv.ProgressBar(len(frame_paths))
det_score_thr=0.5
for frame_path in frame_paths:
    result_det = inference_detector(model, frame_path)
    # We only keep human detections with score larger than det_score_thr
    result = result_det[0][result_det[0][:, 4] >= det_score_thr]
    results.append(result)
    prog_bar.update()

load checkpoint from local path: yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Performing Human Detection for each frame
[                                 ] 2/715, 0.6 task/s, elapsed: 3s, ETA:  1240s

c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 715/715, 14.1 task/s, elapsed: 51s, ETA:     0s

In [14]:
image = cv2.imread(frame_paths[0])
image_shape = (image.shape[0], image.shape[1])
pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths, results)

load checkpoint from local path: hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 715/715, 7.0 task/s, elapsed: 102s, ETA:     0s

In [63]:
pose_results[0,5]

array([[6.0100024e+02, 1.4139496e+02, 8.3008528e-01],
       [6.0450128e+02, 1.3789391e+02, 8.4443128e-01],
       [6.0100024e+02, 1.3614337e+02, 8.1894863e-01],
       [6.0450128e+02, 1.3789391e+02, 7.2430146e-01],
       [5.9049707e+02, 1.3089180e+02, 8.2156581e-01],
       [5.9574866e+02, 1.5014760e+02, 7.6426721e-01],
       [5.6774023e+02, 1.4314548e+02, 8.3695102e-01],
       [6.0625183e+02, 1.7640550e+02, 7.9038960e-01],
       [5.6423920e+02, 1.8340761e+02, 7.3244119e-01],
       [6.2200659e+02, 1.9916237e+02, 5.8502328e-01],
       [5.9049707e+02, 1.6415181e+02, 8.0931008e-01],
       [5.7124127e+02, 1.9916237e+02, 7.0591259e-01],
       [5.5373602e+02, 1.9216023e+02, 7.5521076e-01],
       [5.6774023e+02, 2.3242238e+02, 8.0624080e-01],
       [5.4148230e+02, 2.2892131e+02, 7.2118533e-01],
       [5.4498340e+02, 2.6918341e+02, 8.3252954e-01],
       [5.2222656e+02, 2.6568237e+02, 7.4004245e-01]], dtype=float32)

In [73]:
a=pose_results[...,:2]
a[0,5]

array([[601.00024, 141.39496],
       [604.5013 , 137.8939 ],
       [601.00024, 136.14337],
       [604.5013 , 137.8939 ],
       [590.4971 , 130.8918 ],
       [595.74866, 150.1476 ],
       [567.74023, 143.14548],
       [606.25183, 176.4055 ],
       [564.2392 , 183.40761],
       [622.0066 , 199.16237],
       [590.4971 , 164.15181],
       [571.2413 , 199.16237],
       [553.736  , 192.16023],
       [567.74023, 232.42238],
       [541.4823 , 228.92131],
       [544.9834 , 269.1834 ],
       [522.22656, 265.68237]], dtype=float32)

In [7]:

# files=glob.glob('VIDEO_TEST/*.mp4')
# file='VIDEO_TEST/VIDEO11.mp4'
anno_train = []
# for file in files:
print('Processing ' + files)
anno = pose_extraction(files, det_config, det_checkpoint,pose_config,pose_checkpoint,label=0) #LABEL NOT_FALL
anno_train.append(anno)

Processing VIDEO_TEST/VIDEO1.mp4
load checkpoint from local path: yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Performing Human Detection for each frame
[                                 ] 2/715, 0.4 task/s, elapsed: 6s, ETA:  2030s

c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 715/715, 12.3 task/s, elapsed: 58s, ETA:     0sload checkpoint from local path: hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 715/715, 8.2 task/s, elapsed: 87s, ETA:     0s

In [24]:
anno['keypoint'][0,5]

array([[601.00024, 141.39496],
       [604.5013 , 137.8939 ],
       [601.00024, 136.14337],
       [604.5013 , 137.8939 ],
       [590.4971 , 130.8918 ],
       [595.74866, 150.1476 ],
       [567.74023, 143.14548],
       [606.25183, 176.4055 ],
       [564.2392 , 183.40761],
       [622.0066 , 199.16237],
       [590.4971 , 164.15181],
       [571.2413 , 199.16237],
       [553.736  , 192.16023],
       [567.74023, 232.42238],
       [541.4823 , 228.92131],
       [544.9834 , 269.1834 ],
       [522.22656, 265.68237]], dtype=float32)

In [10]:
mmcv.dump(anno_train, 'train.pkl')
import pandas as pds
data=pds.read_pickle('train.pkl')

In [32]:
data[0]['keypoint'][0,:,:,:].shape

(715, 17, 2)